In [11]:
import requests
import openai
import os
import pandas as pd
from pprint import pprint

In [2]:
# In a Jupyter cell, run:
import sys
print(sys.executable)

/Users/harsh/miniforge3/bin/python


In [3]:
base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

In [4]:
import re
from typing import Dict, Optional

def structured_output(llm_response: str) -> Dict[str, Optional[str]]:
    def extract(tag: str) -> Optional[str]:
        pattern = rf"<{tag}>(.*?)</{tag}>"
        match = re.search(pattern, llm_response, re.S | re.I)
        return match.group(1).strip() if match else None
    
    return {
        "thought": extract("thought"),               # optional, unstable
        "thought_summary": extract("THOUGHT_SUMMARY"),  # preferred
        "final_answer": extract("FINAL"),             # required
    }
out = structured_output(response.choices[0].message.content)
if not out["final_answer"]:
    raise ValueError("No <FINAL> section found in LLM output")
pprint(structured_output(response.choices[0].message.content))

NameError: name 'response' is not defined

In [ ]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "fetch_price",
      "description": "Get the current price of a security, just pass in the ticker symbol.",
      "parameters": {
        "type": "object",
        "properties": {
          "symbol": {
            "type": "string",
            "description": "The ticker symbol of the security being fetched, just pass in the ticker symbol and not the full name e.g. INFY, RELIANCE ",
          }
        },
        "required": ["symbol"],
      },
    }
  }
]

In [ ]:
from pydantic import BaseModel, Field

class GetFetchPriceArgs(BaseModel):
    symbol: str = Field(...,description="The ticker symbol of the security being fetched, just pass in the ticker symbol and not the full name e.g. INFY, RELIANCE ")

schema = [{
    'type':'function',
    "function":{
        "name":"fetch_price",
        "description":"Get the current price of a security, just pass in the ticker symbol.",
        "parameters": GetFetchPriceArgs.model_json_schema()
    }
}
]

In [ ]:
response = response = client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=[
        {
            "role": "user",
            "content": (
                "What is the stock price of hdfc bank?"
            )
        }
    ],
    tools= schema,         #type: ignore
    tool_choice= 'auto'
)

print(response.choices[0].message.content)

None


In [ ]:
pprint(response.choices[0].message.tool_calls)

[ChatCompletionMessageFunctionToolCall(id='function-call-11965070616874333571', function=Function(arguments='{"symbol":"HDFCBANK"}', name='fetch_price'), type='function')]


In [ ]:
pprint(response.choices[0].message.tool_calls[0].function.arguments)

'{"symbol":"HDFCBANK"}'


In [ ]:
from typing import Any
import json

class Agent:
    def __init__(self, client,system:str="",tools:list = None) -> None :
        self.client = client
        self.system = system
        self.messages : list = []
        self.tools = tools if tools is not None else []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self,message = "", *args: Any, **kwds: Any) -> Any:
        if message:
            self.messages.append({"role": "user", "content": message})
        
        final_assistant_content = self.execute()
        
        if message:
            self.messages.append({"role": "assistant", "content": final_assistant_content})

        return final_assistant_content
    
    def execute(self):
        while True:
            completion = self.client.chat.completions.create(
                model="gemini-2.5-flash",
                messages = self.messages,
                tools = self.tools,
                tool_choice = "auto"
            )
            response_message = completion.choices[0].message

            if response_message.tool_calls:
                self.messages.append(response_message)

                tool_outputs = []

                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)

                    if function_name in globals() and callable(globals()[function_name]):
                        function_to_call = globals()[function_name]
                        exected_output = function_to_call(**function_args)
                        print(exected_output)
                        tool_output_content = str(exected_output)
                        print(f"Executing tool:{function_name} with args {function_args}, Output:{tool_output_content[:500]}") #For Debugging

                        tool_outputs.append(
                            {
                                "tool_call_id":tool_call.id,
                                "role":"tool",
                                "name":function_name,
                                "content":tool_output_content
                            }
                        )
                self.messages.extend(tool_outputs)
            
            else:
                return response_message.content
        

In [ ]:
client = openai.OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url=base_url
)
system  = "You are helpful AI assistant."
tools = schema
agent = Agent(client, system, tools)

In [ ]:
agent("What is the price of Infosys stock?")

'\n'

In [ ]:
from nsepython import nsefetch
def fetch_price(symbol: str):
    """
    Get the current price of a security, just pass in the ticker symbol.
    """
    try:
        # Get quote from NSE directly
        url = f"https://www.nseindia.com/api/quote-equity?symbol={symbol}"
        data = nsefetch(url)

        price, percent_change, pe_ratio = (
            data["priceInfo"]["lastPrice"],
            data["priceInfo"]["pChange"],
            data['metadata']['pdSymbolPe']
        )
        return round(price, 2), round(percent_change, 2)
    except Exception as e:
        print(f"Error: {e}")
        return None

In [30]:
url = "https://www.nseindia.com/api/quote-equity?symbol=INFY"
data = nsefetch(url)

In [38]:
data['info']['industry']

'Computers - Software & Consulting'

In [23]:
# Set options to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# Optionally set max column width
pd.set_option('display.max_colwidth', None)

In [ ]:
data = pd.DataFrame(data)

In [28]:
data

,info,metadata,securityInfo,sddDetails,currentMarketType,priceInfo,industryInfo,preOpenMarket
symbol,INFY,INFY,NaN,NaN,NM,NaN,NaN,NaN
companyName,Infosys Limited,NaN,NaN,NaN,NM,NaN,NaN,NaN
industry,Computers - Software & Consulting,Computers - Software & Consulting,NaN,NaN,NM,NaN,IT - Software,NaN
activeSeries,"[EQ, T0]",NaN,NaN,NaN,NM,NaN,NaN,NaN
debtSeries,[],NaN,NaN,NaN,NM,NaN,NaN,NaN
isFNOSec,True,NaN,NaN,NaN,NM,NaN,NaN,NaN
isCASec,False,NaN,NaN,NaN,NM,NaN,NaN,NaN
isSLBSec,True,NaN,NaN,NaN,NM,NaN,NaN,NaN
isDebtSec,False,NaN,NaN,NaN,NM,NaN,NaN,NaN
isSuspended,False,NaN,NaN,NaN,NM,NaN,NaN,NaN


In [29]:
data['pdSymbolPe']

KeyError: 'pdSymbolPe'